# Training and deploying a tabular model using Vertex custom training job - Part 2

![Training pipeline](../images/custom-tabular.png)

In [11]:
import os
import pprint
import pandas as pd
import tensorflow as tf
import time

import matplotlib.pyplot as plt

from google.cloud import aiplatform as vertex_ai
from google.cloud.aiplatform_v1beta1 import types
from google.cloud import bigquery
from google.cloud import exceptions

## Configure GCP settings

*Before running the notebook make sure to follow the repo's README file to install the pre-requisites.*

In [12]:
PROJECT = 'jk-vertex-demos'
REGION = 'us-central1'

STAGING_BUCKET = 'gs://jk-vertex-demos-bucket'
VERTEX_SA = f'training-sa@{PROJECT}.iam.gserviceaccount.com'

BQ_DATASET_NAME = 'chicago_taxi_dataset' 
BQ_TRAIN_SPLIT_NAME = 'training'
BQ_VALID_SPLIT_NAME = 'validation'
BQ_TEST_SPLIT_NAME = 'testing'
BQ_LOCATION = 'US'

## Deploying a model to Vertex AI

### Initialize Vertex AI SDK

In [13]:
vertex_ai.init(
    project=PROJECT,
    location=REGION,
    staging_bucket=STAGING_BUCKET
)

### List custom jobs

In [14]:
job_name = 'CUSTOM_SCRIPT_20210622_161143'
search_filter = 'display_name="{}"'.format(job_name)

for job in vertex_ai.CustomJob.list(filter=search_filter, order_by='create_timedesc'):
    print(job.display_name, job.resource_name, job.state, job.create_time)

CUSTOM_SCRIPT_20210622_161143 projects/1026026909625/locations/us-central1/customJobs/7606971196768256000 JobState.JOB_STATE_SUCCEEDED 2021-06-22 16:11:44.377314+00:00


### Wait for the oldest custom job to complete

In [15]:
while True:
    job_state = job.state
    if job_state != vertex_ai.gapic.JobState.JOB_STATE_SUCCEEDED:
        print("Job has not completed:", job_state)
        if (job_state == vertex_ai.gapic.JobState.JOB_STATE_FAILED or 
           job_state == vertex_ai.gapic.JobState.JOB_STATE_CANCELLED):
            break
    else:
        print("Job has completed")
        break
    time.sleep(60)

Job has completed


### Retrieve model artifacts

In [16]:
saved_model_path = '{}/model'.format(job.job_spec.base_output_directory.output_uri_prefix)

In [17]:
!gsutil ls {saved_model_path}

gs://jk-vertex-demos-bucket/jobs/CUSTOM_SCRIPT_20210622_161143/model/
gs://jk-vertex-demos-bucket/jobs/CUSTOM_SCRIPT_20210622_161143/model/saved_model.pb
gs://jk-vertex-demos-bucket/jobs/CUSTOM_SCRIPT_20210622_161143/model/assets/
gs://jk-vertex-demos-bucket/jobs/CUSTOM_SCRIPT_20210622_161143/model/variables/


### Inspect the model

In [18]:
!saved_model_cli show --dir {saved_model_path} --tag_set serve --signature_def serving_default

2021-06-22 16:45:12.494004: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
The given SavedModel SignatureDef contains the following input(s):
  inputs['dropoff_grid'] tensor_info:
      dtype: DT_STRING
      shape: (-1, 1)
      name: serving_default_dropoff_grid:0
  inputs['euclidean'] tensor_info:
      dtype: DT_DOUBLE
      shape: (-1, 1)
      name: serving_default_euclidean:0
  inputs['payment_type'] tensor_info:
      dtype: DT_STRING
      shape: (-1, 1)
      name: serving_default_payment_type:0
  inputs['pickup_grid'] tensor_info:
      dtype: DT_STRING
      shape: (-1, 1)
      name: serving_default_pickup_grid:0
  inputs['trip_day'] tensor_info:
      dtype: DT_INT64
      shape: (-1, 1)
      name: serving_default_trip_day:0
  inputs['trip_day_of_week'] tensor_info:
      dtype: DT_INT64
      shape: (-1, 1)
      name: serving_default_trip_day_of_week:0
  inputs['trip_hour'] tensor_info:
      dtype:

### Upload the model

In [20]:
display_name = 'Taxi tip classifier'
description = 'Taxi tip TensorFlow classifier'
serving_image_uri = 'us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-4:latest'

model = vertex_ai.Model.upload(
    display_name=display_name,
    description=description,
    artifact_uri=saved_model_path,
    serving_container_image_uri=serving_image_uri
)

model.wait()

INFO:google.cloud.aiplatform.models:Creating Model
INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/1026026909625/locations/us-central1/models/8976527278372552704/operations/4352918430731468800
INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/1026026909625/locations/us-central1/models/8976527278372552704
INFO:google.cloud.aiplatform.models:To use this Model in another session:
INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/1026026909625/locations/us-central1/models/8976527278372552704')


### Create an endpoint

In [21]:
display_name = f'Taxi tip classifier endpoint'

endpoint = vertex_ai.Endpoint.create(display_name=display_name)

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/1026026909625/locations/us-central1/endpoints/6245701034790879232/operations/5433782341300387840
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/1026026909625/locations/us-central1/endpoints/6245701034790879232
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/1026026909625/locations/us-central1/endpoints/6245701034790879232')


### Deploy the Model

In [22]:
deployed_model_display_name = 'taxi-classifier-v1'
traffic_percentage = 100
machine_type = 'n1-standard-4'

endpoint = model.deploy(
        endpoint=endpoint,
        deployed_model_display_name=deployed_model_display_name,
        machine_type=machine_type,
        traffic_percentage=traffic_percentage)


INFO:google.cloud.aiplatform.models:Deploying model to Endpoint : projects/1026026909625/locations/us-central1/endpoints/6245701034790879232
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/1026026909625/locations/us-central1/endpoints/6245701034790879232/operations/6784862229511536640
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/1026026909625/locations/us-central1/endpoints/6245701034790879232


## Invoking the deployed model using Vertex SDK

### Get the endpoint

In [24]:
filter = f'display_name="Taxi tip classifier endpoint"'

for endpoint_info in vertex_ai.Endpoint.list(filter=filter):
    print(endpoint_info)
    
endpoint = vertex_ai.Endpoint(endpoint_info.resource_name)

resource name: projects/1026026909625/locations/us-central1/endpoints/6245701034790879232


### Call the endpoint

In [25]:
test_instances = [  
    
    {
        "dropoff_grid": ["POINT(-87.6 41.9)"],
        "euclidean": [2064.2696],
        "payment_type": ["Credit Card"],
        "pickup_grid": ["POINT(-87.6 41.9)"],
        "trip_miles": [1.37],
        "trip_day": [12],
        "trip_hour": [16],
        "trip_month": [2],
        "trip_day_of_week": [4],
        "trip_seconds": [555]
    }
]

predictions = endpoint.predict(instances=test_instances)
prob = tf.nn.sigmoid(predictions[0])
print('Probability of tip > 20%:', prob.numpy())

Probability of tip > 20%: [[0.7797195]]


## Clean up

### Undeploy models

In [47]:
endpoint.list_models()

[id: "7622184039650164736"
model: "projects/910094146258/locations/us-central1/models/7226878823139115008"
display_name: "jk1-taxi-v1"
create_time {
  seconds: 1624333147
  nanos: 505658000
}
dedicated_resources {
  machine_spec {
    machine_type: "n1-standard-4"
  }
  min_replica_count: 1
  max_replica_count: 1
}
]

In [48]:
endpoint.undeploy_all()

INFO:google.cloud.aiplatform.models:Undeploying Endpoint model: projects/910094146258/locations/us-central1/endpoints/4297331245999718400
INFO:google.cloud.aiplatform.models:Undeploy Endpoint model backing LRO: projects/910094146258/locations/us-central1/endpoints/4297331245999718400/operations/4164083905730707456
INFO:google.cloud.aiplatform.models:Endpoint model undeployed. Resource name: projects/910094146258/locations/us-central1/endpoints/4297331245999718400


resource name: projects/910094146258/locations/us-central1/endpoints/4297331245999718400

### Delete endpoint

In [49]:
endpoint.delete()

INFO:google.cloud.aiplatform.base:Deleting Endpoint : projects/910094146258/locations/us-central1/endpoints/4297331245999718400
INFO:google.cloud.aiplatform.base:Delete Endpoint  backing LRO: projects/910094146258/locations/us-central1/operations/2554047038945755136
INFO:google.cloud.aiplatform.base:Endpoint deleted. . Resource name: projects/910094146258/locations/us-central1/endpoints/4297331245999718400


### Delete model

In [50]:
model.delete()

INFO:google.cloud.aiplatform.base:Deleting Model : projects/910094146258/locations/us-central1/models/7226878823139115008
INFO:google.cloud.aiplatform.base:Delete Model  backing LRO: projects/910094146258/locations/us-central1/operations/3610141151564136448
INFO:google.cloud.aiplatform.base:Model deleted. . Resource name: projects/910094146258/locations/us-central1/models/7226878823139115008
